# Recommender Systems 2024/25

### Practice 9 - Deep Learning Models

## The basics of Deep Learning: Multi-Layer Perceptron 

In [1]:
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Data_manager.Movielens.Movielens10MReader import Movielens10MReader

data_reader = Movielens10MReader()
data_loaded = data_reader.load_data()

URM_all = data_loaded.get_URM_all()

URM_train_val, URM_test = split_train_in_two_percentage_global_sample(URM_all, 0.8)
URM_train, URM_val = split_train_in_two_percentage_global_sample(URM_train_val, 0.8)

Movielens10M: Verifying data consistency...
Movielens10M: Verifying data consistency... Passed!
DataReader: current dataset is: Movielens10M
	Number of items: 10681
	Number of users: 69878
	Number of interactions in URM_all: 10000054
	Value range in URM_all: 0.50-5.00
	Interaction density: 1.34E-02
	Interactions per user:
		 Min: 2.00E+01
		 Avg: 1.43E+02
		 Max: 7.36E+03
	Interactions per item:
		 Min: 0.00E+00
		 Avg: 9.36E+02
		 Max: 3.49E+04
	Gini Index: 0.57

	ICM name: ICM_tags, Value range: 1.00 / 69.00, Num features: 10106, feature occurrences: 106820, density 9.90E-04
	ICM name: ICM_genres, Value range: 1.00 / 1.00, Num features: 20, feature occurrences: 21564, density 1.01E-01
	ICM name: ICM_all, Value range: 1.00 / 69.00, Num features: 10126, feature occurrences: 128384, density 1.19E-03
	ICM name: ICM_year, Value range: 1.92E+03 / 2.01E+03, Num features: 1, feature occurrences: 10681, density 1.00E+00




In [2]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim

if torch.backends.mps.is_available(): # if torch.cuda.is_available() if you use NVIDIA GPUs
    device = torch.device("mps")
else:
    device = torch.device("cpu")

In [3]:
# load iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# create a custom dataset class
class IrisDataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [4]:
# create a custom nn.Module class
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(4, 16)
        self.fc2 = nn.Linear(16, 32)
        self.fc3 = nn.Linear(32, 3)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [5]:
# create a data loader and model
dataset = IrisDataset(X_train, y_train)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)
model = MLP()

# define a loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# train the model
for epoch in range(100):
    running_loss = 0.0
    for i, data in enumerate(data_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print('Epoch %d, loss: %.3f' % (epoch+1, running_loss/(i+1)))

# evaluate the model
model.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for data in data_loader:
        inputs, labels = data
        outputs = model(inputs)
        test_loss += criterion(outputs, labels).item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()

accuracy = correct / len(dataset)
print('Test loss: {:.3f}, Accuracy: {:.2f}'.format(test_loss/(len(data_loader)), accuracy*100))

Epoch 1, loss: 1.171
Epoch 2, loss: 1.134
Epoch 3, loss: 1.108
Epoch 4, loss: 1.086
Epoch 5, loss: 1.066
Epoch 6, loss: 1.052
Epoch 7, loss: 1.037
Epoch 8, loss: 1.024
Epoch 9, loss: 1.013
Epoch 10, loss: 0.998
Epoch 11, loss: 0.984
Epoch 12, loss: 0.971
Epoch 13, loss: 0.958
Epoch 14, loss: 0.945
Epoch 15, loss: 0.933
Epoch 16, loss: 0.920
Epoch 17, loss: 0.907
Epoch 18, loss: 0.896
Epoch 19, loss: 0.881
Epoch 20, loss: 0.869
Epoch 21, loss: 0.853
Epoch 22, loss: 0.841
Epoch 23, loss: 0.830
Epoch 24, loss: 0.818
Epoch 25, loss: 0.801
Epoch 26, loss: 0.790
Epoch 27, loss: 0.779
Epoch 28, loss: 0.765
Epoch 29, loss: 0.752
Epoch 30, loss: 0.741
Epoch 31, loss: 0.725
Epoch 32, loss: 0.715
Epoch 33, loss: 0.700
Epoch 34, loss: 0.685
Epoch 35, loss: 0.677
Epoch 36, loss: 0.665
Epoch 37, loss: 0.659
Epoch 38, loss: 0.653
Epoch 39, loss: 0.636
Epoch 40, loss: 0.631
Epoch 41, loss: 0.617
Epoch 42, loss: 0.610
Epoch 43, loss: 0.601
Epoch 44, loss: 0.589
Epoch 45, loss: 0.577
Epoch 46, loss: 0.5

## Two-Tower Models

In [6]:
# Repro '19 on Maurizio's Github – NeuMF: principal components and explain
# https://github.com/MaurizioFD/RecSys2019_DeepLearning_Evaluation/blob/0fb6b7f5c396f8525316ed66cf9c9fdb03a5fa9b/Conferences/WWW/NeuMF_our_interface/NeuMF_RecommenderWrapper.py#L110C33-L110C65
# Rows 108-113
# Need to convert to pytorch
def NeuCF_get_model(num_users, num_items, mf_dim=10, layers=[10], reg_layers=[0], reg_mf=0.0):
    assert len(layers) == len(reg_layers)
    num_layer = len(layers) #Number of layers in the MLP
    # Input variables
    user_input = Input(shape=(1,), dtype='int32', name = 'user_input')
    item_input = Input(shape=(1,), dtype='int32', name = 'item_input')

    # Embedding layer
    MF_Embedding_User = Embedding(input_dim = num_users, output_dim = mf_dim, name = 'mf_embedding_user',
                                  embeddings_initializer = 'random_normal', embeddings_regularizer = l2(reg_mf), input_length=1)
    MF_Embedding_Item = Embedding(input_dim = num_items, output_dim = mf_dim, name = 'mf_embedding_item',
                                  embeddings_initializer = 'random_normal', embeddings_regularizer = l2(reg_mf), input_length=1)

    MLP_Embedding_User = Embedding(input_dim = num_users, output_dim = int(layers[0]/2), name = "mlp_embedding_user",
                                   embeddings_initializer = 'random_normal', embeddings_regularizer = l2(reg_layers[0]), input_length=1)
    MLP_Embedding_Item = Embedding(input_dim = num_items, output_dim = int(layers[0]/2), name = 'mlp_embedding_item',
                                   embeddings_initializer = 'random_normal', embeddings_regularizer = l2(reg_layers[0]), input_length=1)

    # MF part
    mf_user_latent = Flatten()(MF_Embedding_User(user_input))
    mf_item_latent = Flatten()(MF_Embedding_Item(item_input))
    mf_vector = Multiply()([mf_user_latent, mf_item_latent]) # element-wise multiply

    # MLP part
    mlp_user_latent = Flatten()(MLP_Embedding_User(user_input))
    mlp_item_latent = Flatten()(MLP_Embedding_Item(item_input))
    mlp_vector = Concatenate()([mlp_user_latent, mlp_item_latent])
    for idx in range(1, num_layer):
        layer = nn.Dense(layers[idx], kernel_regularizer= l2(reg_layers[idx]), activation='relu', name="layer%d" %idx)
        mlp_vector = layer(mlp_vector)

    # Concatenate MF and MLP parts
    predict_vector = Concatenate()([mf_vector, mlp_vector])

    # Final prediction layer
    prediction = Dense(1, activation='sigmoid', kernel_initializer='lecun_uniform', name = "prediction")(predict_vector)

    model = Model(inputs=[user_input, item_input],
                  outputs=prediction)

    return model


In [7]:
def NeuCF_get_model(num_users, num_items, mf_dim=10, layers=[10], reg_layers=[0], reg_mf=0.0):
    assert len(layers) == len(reg_layers)
    num_layer = len(layers) #Number of layers in the MLP

    class NeuCF(nn.Module):
        def __init__(self, num_users, num_items, mf_dim, layers, reg_layers, reg_mf):
            super(NeuCF, self).__init__()
            self.mf_embedding_user = nn.Embedding(num_users, mf_dim)
            self.mf_embedding_item = nn.Embedding(num_items, mf_dim)
            self.mlp_embedding_user = nn.Embedding(num_users, int(layers[0]/2))
            self.mlp_embedding_item = nn.Embedding(num_items, int(layers[0]/2))

            self.mlp_layers = nn.ModuleList([nn.Linear(int(layers[0]/2)*2, layers[i], bias=True) for i in range(1, num_layer)])
            for i, layer in enumerate(self.mlp_layers):
                nn.init.normal_(layer.weight)
                layer.bias.data.zero_()
                layer.weight_decay = reg_layers[i]

            self.prediction_layer = nn.Linear(mf_dim + layers[-1], 1, bias=True)
            nn.init.lecun_uniform_(self.prediction_layer.weight)
            self.prediction_layer.bias.data.zero_()

        def forward(self, user_input, item_input):
            mf_user_latent = self.mf_embedding_user(user_input)
            mf_item_latent = self.mf_embedding_item(item_input)
            mf_vector = torch.mul(mf_user_latent, mf_item_latent)

            mlp_user_latent = self.mlp_embedding_user(user_input)
            mlp_item_latent = self.mlp_embedding_item(item_input)
            mlp_vector = torch.cat((mlp_user_latent, mlp_item_latent), dim=1)
            for layer in self.mlp_layers:
                mlp_vector = torch.relu(layer(mlp_vector))

            predict_vector = torch.cat((mf_vector, mlp_vector), dim=1)
            prediction = torch.sigmoid(self.prediction_layer(predict_vector))
            return prediction

    model = NeuCF(num_users, num_items, mf_dim, layers, reg_layers, reg_mf)
    return model

In [8]:
import numpy as np

In [9]:
from Recommenders.BaseRecommender import BaseRecommender
import scipy.sparse as sp
# Variant 1:
# 1. Make 2 embeddings of equal dimensions and concatenate
# 2. Couple of Dense layers
# 3. Obtain prediction (single score)
class TwoTowerRecommender_type1(nn.Module, BaseRecommender):
    
    RECOMMENDER_NAME = """TWO_TOWER_1"""
    
    def __init__(self, URM_train, num_users, num_items, layers=[10], reg_layers=[0], verbose = True):
        global device
        super().__init__()
        BaseRecommender.__init__(self, URM_train, verbose=verbose)
        self.device = device
        self.mlp_embedding_user = nn.Embedding(num_users, int(layers[0]/2), device=self.device)
        self.mlp_embedding_item = nn.Embedding(num_items, int(layers[0]/2), device=self.device)

        self.mlp_layers = nn.ModuleList([
            nn.Linear(layers[i-1], layers[i], bias=True, device=self.device) for i in range(1, len(layers))
            ])
        for i, layer in enumerate(self.mlp_layers):
            nn.init.normal_(layer.weight)
            layer.bias.data.zero_()
            layer.weight_decay = reg_layers[i]

        self.prediction_layer = nn.Linear(layers[-1], 1, bias=True, device=self.device)
        nn.init.uniform_(self.prediction_layer.weight)
        self.prediction_layer.bias.data.zero_()
        self.to(self.device)

    def _data_generator(self, batch_size):
        # stupid sampling, will lead to overfit zeros
        row_idx = np.random.choice(self.URM_train.shape[0], batch_size, replace=False).astype(np.int32)
        col_idx = np.random.choice(self.URM_train.shape[1], batch_size, replace=False).astype(np.int32)
        user_input = torch.tensor(row_idx, device=self.device)
        item_input = torch.tensor(col_idx, device=self.device)
        labels = np.zeros(batch_size, dtype=np.int32)
        for i in range(len(labels)):
            if self.URM_train[row_idx[i], col_idx[i]] > 0:
                labels[i] = 1
        labels.shape = (batch_size, 1)
        labels = torch.tensor(labels, device=self.device)
        yield user_input, item_input, labels

    def forward(self, user_input, item_input):
        mlp_user_latent = self.mlp_embedding_user(user_input.long().to(self.device))
        mlp_item_latent = self.mlp_embedding_item(item_input.long().to(self.device))
        mlp_vector = torch.cat((mlp_user_latent, mlp_item_latent), dim=1)
        for layer in self.mlp_layers:
            mlp_vector = torch.relu(layer(mlp_vector))

        predict_vector = mlp_vector
        prediction = torch.sigmoid(self.prediction_layer(predict_vector))
        return prediction
    
    def fit(self, epochs=30, batch_size=1024, learning_rate=0.0001):
        optimizer = torch.optim.Adam(self.parameters(), lr=learning_rate)
        for i in range(epochs):
            for user_input, item_input, labels in self._data_generator(batch_size):
                optimizer.zero_grad()
                predictions = self.forward(user_input, item_input)
                loss = torch.nn.BCELoss().to(self.device)
                loss = loss(predictions, labels.float())
                loss.backward()
                optimizer.step()
            self._print("Epoch {} finished. Loss: {}".format(i, loss.item()))

    def _compute_item_score(self, user_id_array, items_to_compute=None):
        step = user_id_array.shape[0]
        
        if items_to_compute is None:
            items_to_compute = np.arange(self.URM_train.shape[1], dtype=np.int32)
        
        predictions = np.empty((step,items_to_compute.shape[0]))
        for item in items_to_compute:
            predictions[:, item] = self.forward(
                torch.tensor(user_id_array),
                torch.tensor(
                    np.ones(step, dtype=np.int32) * item)
                ).cpu().detach().numpy().ravel()
        return predictions

In [13]:
# Variant 2:
# 1. Couple of Dense layers process user/item profiles
# 2. Concatenate and final Dense layer to obtain prediciton
class TwoTowerRecommender_type2(nn.Module, BaseRecommender):

    RECOMMENDER_NAME = """TWO_TOWER_2"""

    def __init__(self, URM_train, num_users, num_items, layers=[10], reg_layers=[0], verbose = True):
        global device
        super().__init__()
        BaseRecommender.__init__(self, URM_train, verbose=verbose)
        self.device = device
        layers[0] = int(layers[0]/2) # <- The first layer is split in two tower inputs at the beginning
        self.mlp_embedding_user = nn.Embedding(num_users, layers[0], device=self.device)
        self.mlp_embedding_item = nn.Embedding(num_items, layers[0], device=self.device) # <- It's possible to make the towers asymmetric! Mind the output dimension though

        # layer_input_sizes_tower1 = layers
        # layer_input_sizes_tower1[0] = num_items
        self.mlp_layers_tower1 = nn.ModuleList([
            nn.Linear(
                layers[i-1],
                layers[i], bias=True, device=self.device
                ) for i in range(1, len(layers))
            ])
        
        # layer_input_sizes_tower2 = layers
        # layer_input_sizes_tower2[0] = num_users
        self.mlp_layers_tower2 = nn.ModuleList([
            nn.Linear(
                layers[i-1],
                layers[i], bias=True, device=self.device
                ) for i in range(1, len(layers))
            ])
        
        for i, layer in enumerate(self.mlp_layers_tower1):
            nn.init.normal_(layer.weight)
            layer.bias.data.zero_()
            layer.weight_decay = reg_layers[i]

        for i, layer in enumerate(self.mlp_layers_tower2):
            nn.init.normal_(layer.weight)
            layer.bias.data.zero_()
            layer.weight_decay = reg_layers[i]

        self.prediction_layer = nn.Linear(layers[-1], 1, bias=True, device=self.device)
        nn.init.uniform_(self.prediction_layer.weight)
        self.prediction_layer.bias.data.zero_()
        self.to(self.device)

    def _data_generator(self, batch_size):
        # TODO: adjust sampling
        row_idx = np.random.choice(self.URM_train.shape[0], batch_size, replace=False).astype(np.int32)
        col_idx = np.random.choice(self.URM_train.shape[1], batch_size, replace=False).astype(np.int32)
        user_input = torch.tensor(row_idx, device=self.device)
        item_input = torch.tensor(col_idx, device=self.device)
        labels = np.zeros(batch_size, dtype=np.int32)
        for i in range(len(labels)):
            if self.URM_train[row_idx[i], col_idx[i]] > 0:
                labels[i] = 1
        labels.shape = (batch_size, 1) # <- =1 for every user_item couple if an interaction exists, 0 otherwise
        labels = torch.tensor(labels, device=self.device)
        yield user_input, item_input, labels

    def forward(self, user_input, item_input):
        mlp_user_latent = self.mlp_embedding_user(user_input.long().to(self.device))
        mlp_item_latent = self.mlp_embedding_item(item_input.long().to(self.device))

        mlp_user_vector = mlp_user_latent
        mlp_item_vector = mlp_item_latent

        for layer in self.mlp_layers_tower1:
            mlp_user_vector = torch.relu(layer(mlp_user_vector))

        for layer in self.mlp_layers_tower2:
            mlp_item_vector = torch.relu(layer(mlp_item_vector))

        predict_vector = mlp_user_vector * mlp_item_vector # <- Merge the tensors via element-wise multiplication
        prediction = torch.sigmoid(self.prediction_layer(predict_vector))
        return prediction
    
    def fit(self, epochs=30, batch_size=1024, learning_rate=0.0001):
        optimizer = torch.optim.Adam(self.parameters(), lr=learning_rate)
        for i in range(epochs):
            for user_input, item_input, labels in self._data_generator(batch_size):
                optimizer.zero_grad()
                predictions = self.forward(user_input, item_input)
                loss = torch.nn.BCELoss().to(self.device)
                loss = loss(predictions, labels.float())
                loss.backward()
                optimizer.step()
            self._print("Epoch {} finished. Loss: {}".format(i, loss.item()))
            torch.mps.empty_cache() # <- Input embeddings are pretty huge this time around...

    def _compute_item_score(self, user_id_array, items_to_compute=None):
        step = user_id_array.shape[0]
        
        if items_to_compute is None:
            items_to_compute = np.arange(self.URM_train.shape[1], dtype=np.int32)
        
        predictions = np.empty((step,items_to_compute.shape[0]))
        for item in items_to_compute:
            predictions[:, item] = self.forward(
                torch.tensor(user_id_array),
                torch.tensor(
                    np.ones(step, dtype=np.int32) * item)
                ).cpu().detach().numpy().ravel()
        return predictions

In [11]:
# Training and testing
from Evaluation.Evaluator import EvaluatorHoldout

evaluator_test = EvaluatorHoldout(URM_test, [10])
evaluator_validation = EvaluatorHoldout(URM_val, [10])

EvaluatorHoldout: Ignoring 73 ( 0.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 252 ( 0.4%) Users that have less than 1 test interactions


In [12]:
# Train and test type 1
twotower_1 = TwoTowerRecommender_type1(URM_train, URM_train.shape[0], URM_train.shape[1], layers=[10,5,2,2], reg_layers=[0,0,0,0])

twotower_1.fit(epochs=300, batch_size=1024, learning_rate=0.01)

results_df, _ = evaluator_test.evaluateRecommender(twotower_1)

results_df

TWO_TOWER_1: URM Detected 50 ( 0.5%) items with no interactions.
TWO_TOWER_1: Epoch 0 finished. Loss: 1.6019437313079834
TWO_TOWER_1: Epoch 1 finished. Loss: 1.5645570755004883
TWO_TOWER_1: Epoch 2 finished. Loss: 1.3969138860702515
TWO_TOWER_1: Epoch 3 finished. Loss: 1.3517330884933472
TWO_TOWER_1: Epoch 4 finished. Loss: 1.31689453125
TWO_TOWER_1: Epoch 5 finished. Loss: 1.1823232173919678
TWO_TOWER_1: Epoch 6 finished. Loss: 1.1204612255096436
TWO_TOWER_1: Epoch 7 finished. Loss: 1.1037973165512085
TWO_TOWER_1: Epoch 8 finished. Loss: 1.0747756958007812
TWO_TOWER_1: Epoch 9 finished. Loss: 0.9497035145759583
TWO_TOWER_1: Epoch 10 finished. Loss: 0.9518728256225586
TWO_TOWER_1: Epoch 11 finished. Loss: 0.910079836845398
TWO_TOWER_1: Epoch 12 finished. Loss: 0.8842508792877197
TWO_TOWER_1: Epoch 13 finished. Loss: 0.8474003672599792
TWO_TOWER_1: Epoch 14 finished. Loss: 0.8092023134231567
TWO_TOWER_1: Epoch 15 finished. Loss: 0.7797098159790039
TWO_TOWER_1: Epoch 16 finished. Loss: 0

,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.005035,0.005326,0.001844,0.001481,0.001561,0.013827,0.002833,0.002699,0.047228,0.014309,...,0.999027,0.047182,0.999027,0.028017,8.511292,0.995388,0.143894,0.75155,0.150539,0.144838


In [22]:
torch.mps.empty_cache()

In [14]:
# Train and test type 2
twotower_2 = TwoTowerRecommender_type2(URM_train, URM_train.shape[0], URM_train.shape[1], layers=[10,5,2,2], reg_layers=[0,0,0,0])

twotower_2.fit(epochs=100, batch_size=1024, learning_rate=0.005)

results_df, _ = evaluator_test.evaluateRecommender(twotower_2)

results_df

TWO_TOWER_2: URM Detected 64 ( 0.6%) items with no interactions.
TWO_TOWER_2: Epoch 0 finished. Loss: 0.7134952545166016
TWO_TOWER_2: Epoch 1 finished. Loss: 0.7043734788894653
TWO_TOWER_2: Epoch 2 finished. Loss: 0.7000753879547119
TWO_TOWER_2: Epoch 3 finished. Loss: 0.6923127174377441
TWO_TOWER_2: Epoch 4 finished. Loss: 0.6904811859130859
TWO_TOWER_2: Epoch 5 finished. Loss: 0.6877688765525818
TWO_TOWER_2: Epoch 6 finished. Loss: 0.6841063499450684
TWO_TOWER_2: Epoch 7 finished. Loss: 0.6899026036262512
TWO_TOWER_2: Epoch 8 finished. Loss: 0.6785768866539001
TWO_TOWER_2: Epoch 9 finished. Loss: 0.6749589443206787
TWO_TOWER_2: Epoch 10 finished. Loss: 0.6726386547088623
TWO_TOWER_2: Epoch 11 finished. Loss: 0.6690514087677002
TWO_TOWER_2: Epoch 12 finished. Loss: 0.6679033041000366
TWO_TOWER_2: Epoch 13 finished. Loss: 0.6649109125137329
TWO_TOWER_2: Epoch 14 finished. Loss: 0.6612294912338257
TWO_TOWER_2: Epoch 15 finished. Loss: 0.6584392786026001
TWO_TOWER_2: Epoch 16 finished. L

## AutoEncoders

### Denoising Autoencoder

In [ ]:
from Recommenders.MatrixFactorization.PyTorch import MF_MSE_PyTorch
# ...
# Autoencoder for recommendation, search on internet
# Couple layers before and couple after bottleneck, basic noise (10 lines of code)

In [ ]:
# Build the encoder

In [ ]:
# Build the decoder

In [ ]:
# Implement the training structure using PyTorch

In [ ]:
# Train and evaluate

### $EASE^R$

In [ ]:
# only import, training and evaluation (from repo)

## LightGCN


In [ ]:
# PyTorch implementation (see repo)

Possibly, GF-CF